# Notes

MLS librispeech doesn't have punctuation... This might be not good for TTS

In [ ]:
from datasets import list_datasets, load_dataset, load_metric, Audio

In [ ]:
dataset = load_dataset('facebook/multilingual_librispeech', 'spanish')

train_dataset = dataset['train.9h']
test_dataset = dataset['test']

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_dataset.remove_columns(['file', 'audio', 'chapter_id', 'id']))

In [ ]:
show_random_elements(val_dataset.remove_columns(['file', 'audio', 'chapter_id', 'id']))

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\!\¿\¡\!\?]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

In [ ]:
show_random_elements(train_dataset.remove_columns(['file', 'audio', 'chapter_id', 'id']))

In [ ]:
show_random_elements(val_dataset.remove_columns(['file', 'audio', 'chapter_id', 'id']))

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train_dataset.map(
    extract_all_chars, batched=True,
    batch_size=-1, keep_in_memory=True,
    remove_columns=train_dataset.column_names
)
vocab_val = val_dataset.map(
  extract_all_chars, batched=True, 
  batch_size=-1, keep_in_memory=True, 
  remove_columns=train_dataset.column_names
)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_val["vocab"][0]))
vocab_dict = {v: k for k,v in enumerate(vocab_list)}

vocab_dict['|'] = vocab_dict[' ']
del vocab_dict[' ']

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

In [ ]:
from IPython.display import Audio

rand_int = random.randint(0, len(train_dataset)-1)

print(train_dataset[rand_int]["text"])
Audio(data=train_dataset[rand_int]["audio"]["array"], autoplay=True, rate=16000)